## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


### (Advanced Users) Installing additional dependencies

Here we show you how to install additional packages into this Deepnote project. We will use the package [gcsopt](https://pypi.org/project/gcsopt/) as an example. To install additional dependencies, you have two choices:
- **Recommended** (automatically installs your dependencies every time you boot the Deepnote machine):
  1. Add the dependency to the list of dependencies in `reqirements.txt` on the left.  
      - **Note**: Make sure to NOT remove the text that is already in the file. You should add your dependencies on new lines AFTER `manipulation=year.month.day`. For our example, the file contents will look like:
          ```
          --extra-index-url https://drake-packages.csail.mit.edu/whl/nightly
          manipulation==2025.9.23
          gcsopt
          ```
  2. Restart the Deepnote machine (or run the `Init` notebook)
- **Alternatively:** Run `!pip install gcsopt` within a cell in the Deepnote notebook. You will have to do this everytime you restart your Deepnote machine.

### (Advanced Users) Updating the `manipulation` version
We continually add new features to the `manipulation` package, which is automatically updated in Deepnote. However, after you first click "Duplicate", you will stop receiving these updates. Follow these steps to update your version to the latest version:
1. Find the latest version number for `manipulation` on the [PyPi package page](https://pypi.org/project/manipulation/#history).
2. Update the version number for `manipulation=year.month.day` in the `requirements.txt` file on the left
3. Restart the Deepnote machine (or run the `Init` notebook)

The changes will persist every time you start your Deepnote machine.

Good luck!

---


## Placeholder code: iiwa Simulation

As a starting point, the code below sets up a simple simulation with a single iiwa, visualized in Meshcat.

### Setup and imports
Let us first get our imports out of the way:

In [1]:
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    BasicVector,
    LeafSystem,
    InverseKinematics,
    SnoptSolver,
    RotationMatrix,
    RigidTransform,
    Solve,
    LogVectorOutput,
    PiecewisePolynomial,
    TrajectorySource,
    ImageWriter,
    PixelType
)

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.geometry import Cylinder, Rgba

import matplotlib.pyplot as plt
import numpy as np
import pygame
import math
import os
import h5py
import numpy as np
import collections

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7004


Click the link above to open Meshcat in your browser!


### Setting up the Scenario

In [3]:
scenario_string = """directives:
# add robot
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

# add gripper
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

# add camera mounted to world 
- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [270, 0.0, 90.0]}
        translation: [1.2, -0.5, 0.5]
- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera0_origin
    child: camera0::base

# add camera mounted to robot wrist
- add_frame:
    name: camera_wrist
    X_PF:
      base_frame: iiwa::iiwa_link_7
      translation: [-0.05, 0, 0.1]   # 10 cm ahead of wrist
      rotation: !Rpy {deg: [0, 00, -90]}
- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: iiwa::camera_wrist
    child: camera1::base

# add cabinet
- add_model:
    name: cabinet
    file: package://drake_models/manipulation_station/cupboard.sdf
- add_frame:
    name: cabinet_origin
    X_PF:
      base_frame: world
      translation: [0, -1.1, 0.4]    # x, y, z in meters
      rotation: !Rpy { deg: [0, 0, 90]}  # roll, pitch, yaw
- add_weld:
    parent: cabinet_origin
    child: cabinet::cupboard_body

cameras:
  camera0:
    name: camera0
    depth: True
    X_PB:
      base_frame: camera0::base
  camera1:
    name: camera1
    depth: True
    X_PB:
      base_frame: camera1::base

model_drivers:
  iiwa: !IiwaDriver
    control_mode: position_only 
    hand_model_name: wsg
  wsg: !SchunkWsgDriver {}
"""

In [4]:
import ctypes
import numpy as np

# -------- Spacenav / SpaceMouse bindings via libspnav --------

libspnav = ctypes.CDLL("libspnav.so")

SPNAV_EVENT_MOTION = 1
SPNAV_EVENT_BUTTON = 2

class SpnavMotion(ctypes.Structure):
    _fields_ = [
        ("x", ctypes.c_int),
        ("y", ctypes.c_int),
        ("z", ctypes.c_int),
        ("rx", ctypes.c_int),
        ("ry", ctypes.c_int),
        ("rz", ctypes.c_int),
    ]

class SpnavButton(ctypes.Structure):
    _fields_ = [
        ("bnum", ctypes.c_int),
        ("press", ctypes.c_int),
    ]

class SpnavEvent(ctypes.Structure):
    _fields_ = [
        ("type", ctypes.c_int),
        ("motion", SpnavMotion),
        ("button", SpnavButton),
    ]


class SpaceMouse:
    """Thin wrapper around libspnav using ctypes."""

    def __init__(self):
        rc = libspnav.spnav_open()
        if rc == -1:
            print("ERROR: Could not connect to spacenavd. Is it running?")
            self.active = False
        else:
            print("SpaceMouse connected via libspnav!")
            self.active = True

        # Reusable event struct to avoid reallocations
        self._event = SpnavEvent()

    def poll_event(self):
        """Non-blocking poll. Returns SpnavEvent or None."""
        if not self.active:
            return None
        n = libspnav.spnav_poll_event(ctypes.byref(self._event))
        if n > 0:
            # copy to avoid overwriting by next poll
            ev_copy = SpnavEvent()
            ctypes.pointer(ev_copy)[0] = self._event
            return ev_copy
        return None

    def close(self):
        if self.active:
            libspnav.spnav_close()
            self.active = False


In [15]:
class TeleopController(LeafSystem):
    """
    Buttons for end-effector pose to joint positions via IK, using SpaceMouse.
    """
    def __init__(self, meshcat, plant):
        LeafSystem.__init__(self)
        
        # --- Spacemouse initialization ---
        self.spacemouse = SpaceMouse()
        self.spacemouse_active = self.spacemouse.active

        # Scale factors
        self.TRANSLATION_GAIN = 0.00001     # meters per unit input
        self.ROTATION_GAIN    = 0.00001     # rad per unit input

        # Low-pass filter coefficient
        self.has_new_input = False  # Track if we have new input from SpaceMouse
        self.DEADZONE_THRESHOLD = 1e-7  # Ignore movements smaller than this

        # --- Drake setup ---
        self.meshcat = meshcat
        self.plant = plant
        
        # getting the necessary frames
        self.plant_context = plant.CreateDefaultContext()
        self.W_frame = plant.world_frame()

        wsg_instance = self.plant.GetModelInstanceByName("wsg")
        wsg_link = self.plant.GetBodyByName("body", wsg_instance)
        self.WG_frame = wsg_link.body_frame()
        
        self.nq = 7 
        
        # Configuration for IK
        self.q_initial = np.array([-1.57, 0.2, 0, -1.2, 0, 1.6, 0])
        self.iiwa = self.plant.GetModelInstanceByName("iiwa")
        first_iiwa_joint = self.plant.GetJointByName("iiwa_joint_1", self.iiwa)
        self.iiwa_q_start_index = first_iiwa_joint.position_start()
        
        # --- Initialize EE pose and desired joint positions from current plant ---
        self.plant.SetPositions(self.plant_context, self.iiwa, self.q_initial)
        self.q_desired = np.copy(self.q_initial)

        # Sync desired end-effector pose with the actual pose of the gripper
        self.X_WG_desired = self.plant.CalcRelativeTransform(
            self.plant_context, self.W_frame, self.WG_frame
        )
        self.R_WG_desired = self.X_WG_desired.rotation()

        self.wsg_position_desired = 0.1

        # Register output ports for IIWA and WSG commands
        self.DeclareVectorOutputPort(
            "iiwa_position_command", 
            BasicVector(self.nq), 
            self.DoCalcIiwaOutput
        )
        self.DeclareVectorOutputPort(
            "wsg_position_command", 
            BasicVector(1), 
            lambda context, output: output.SetFromVector([self.wsg_position_desired])
        )

        meshcat.AddButton("Stop Simulation")
        
        # Clear any old debug visualizations
        self.meshcat.Delete("ik_debug")


    def read_spacemouse(self):
        """
        Returns a filtered 6D twist vector [vx, vy, vz, wx, wy, wz]
        Only returns non-zero values when there's actual new input.
        """
        if not self.spacemouse_active:
            self.has_new_input = False 
            return np.zeros(6), 0, 0

        event = self.spacemouse.poll_event()
        if event is not None and event.type == SPNAV_EVENT_MOTION:
            twist_raw = np.array([
                self.TRANSLATION_GAIN * event.motion.x,
                self.TRANSLATION_GAIN * event.motion.y,
                self.TRANSLATION_GAIN * event.motion.z,
                self.ROTATION_GAIN * event.motion.rx,
                self.ROTATION_GAIN * event.motion.ry,
                self.ROTATION_GAIN * event.motion.rz
            ])

            # Check if input is significant (deadzone)
            if np.linalg.norm(twist_raw) > self.DEADZONE_THRESHOLD:
                self.has_new_input = True
                return twist_raw, 0, 0

        self.has_new_input = False
        return np.zeros(6), 0, 0


    def step_control(self):
        twist, btn_close, btn_open = self.read_spacemouse()
        vx, vy, vz, wx, wy, wz = twist

        # Clip to maximum per-step values
        MAX_TRANSLATION_STEP = 0.01  # meters per update (reasonable)
        MAX_ROTATION_STEP    = 0.05   # radians per update
        vx, vy, vz = np.clip([vx, vy, vz], -MAX_TRANSLATION_STEP, MAX_TRANSLATION_STEP)
        wx, wy, wz = np.clip([wx, wy, wz], -MAX_ROTATION_STEP, MAX_ROTATION_STEP)

        if not self.has_new_input:
            # Freeze desired pose at current actual pose to stop motion
            X_WG_current = self.plant.CalcRelativeTransform(self.plant_context, self.W_frame, self.WG_frame)
            self.X_WG_desired = X_WG_current
            self.R_WG_desired = X_WG_current.rotation()
            # also update q_desired to current positions so the output matches
            q_current = self.plant.GetPositions(self.plant_context)[self.iiwa_q_start_index:self.iiwa_q_start_index + self.nq]
            self.q_desired = q_current
    
        # Only update if we have new input
        moved = self.has_new_input

        if moved:
            self.X_WG_desired.set_translation(
                self.X_WG_desired.translation() + np.array([vx, vy, vz])
            )
            dR = (
                RotationMatrix.MakeXRotation(wx)
                .multiply(RotationMatrix.MakeYRotation(wy))
                .multiply(RotationMatrix.MakeZRotation(wz))
            )
            self.R_WG_desired = dR.multiply(self.R_WG_desired) ## what's the order of multiplication TODO 
            # self.R_WG_desired = self.R_WG_desired.multiply(dR)
            self.X_WG_desired = RigidTransform(self.R_WG_desired, self.X_WG_desired.translation())

        # Update gripper
        if btn_close == 1:
            self.wsg_position_desired = 0.0
        elif btn_open == 1:
            self.wsg_position_desired = 0.1

        # Only calculate IK if moved
        if moved or btn_close == 1 or btn_open == 1:
            self._calculate_ik()


    def _calculate_ik(self):
        """
        Finds the joint positions (q_desired) that best match the desired
        end-effector pose (X_WG_desired), using the last valid solution as a seed.
        """
        print("finding IK solution...")
        ik = InverseKinematics(self.plant, self.plant_context)
        
        # define decision variables for only the IIWA joints
        q_iiwa = ik.q()[self.iiwa_q_start_index : self.iiwa_q_start_index + self.nq]
        
        p_WG_W = self.X_WG_desired.translation()
        tol = 0.01  # 1 cm
        ik.AddPositionConstraint(
            self.WG_frame, [0,0,0], self.W_frame,
            p_WG_W - tol,
            p_WG_W + tol
        )
        
        R_WG_W = self.X_WG_desired.rotation()
        ik.AddOrientationConstraint(
            self.WG_frame, RotationMatrix(), self.W_frame, R_WG_W, 0.05
        )
        
        prog = ik.prog()
        q_current = self.q_desired
        
        # cost minimizes distance b/w current and future state
        prog.AddQuadraticErrorCost(
            np.identity(self.nq), 
            q_current, 
            q_iiwa
        )
        
        solver = SnoptSolver()
        
        # set initial guess
        q_full_initial_guess = self.plant.GetPositions(self.plant_context)
        q_full_initial_guess[self.iiwa_q_start_index:self.iiwa_q_start_index+self.nq] = self.q_desired

        
        result = solver.Solve(prog, initial_guess=q_full_initial_guess)

        if result.is_success():
            # update positions
            q_full_result = result.GetSolution(ik.q())
            q_iiwa_result = q_full_result[self.iiwa_q_start_index : self.iiwa_q_start_index + self.nq]
            
            self.q_desired = q_iiwa_result
            self.plant.SetPositions(self.plant_context, self.iiwa, self.q_desired)
        else:
            print(f"IK Warning: Failed to find joint solution. Check if target is reachable.")
            # Reset desired pose to current pose to prevent drift
            # Get current pose from the plant context (using current joint positions)
            X_WG_current = self.plant.CalcRelativeTransform(
                self.plant_context, self.W_frame, self.WG_frame
            )
            self.X_WG_desired = X_WG_current
            self.R_WG_desired = self.X_WG_desired.rotation()
            print("Reset X_WG_desired to current pose to prevent drift.")

            
    def DoCalcIiwaOutput(self, context, output):
        """
        Outputs the latest calculated joint position command.
        """
        output.SetFromVector(self.q_desired)
    
    def visualize_frames(self, plant_context):
        """
        Visualizes the desired and current end-effector poses as coordinate frames.
        """
        # Get current X_WG from the plant
        X_WG_current = self.plant.CalcRelativeTransform(
            plant_context, self.W_frame, self.WG_frame
        )
        
        # Visualize desired frame (in red/green/blue, slightly transparent)
        AddMeshcatTriad(
            self.meshcat,
            path="ik_debug/X_WG_desired",
            length=0.15,
            radius=0.01,
            opacity=0.8,
            X_PT=self.X_WG_desired
        )
        
        # Visualize current frame (in brighter colors, fully opaque)
        AddMeshcatTriad(
            self.meshcat,
            path="ik_debug/X_WG_current",
            length=0.15,
            radius=0.01,
            opacity=1.0,
            X_PT=X_WG_current
        )


# Teleopping

In [16]:
WSG_MIN = 0.0        # fully closed width (meters)
WSG_MAX = 0.1        # fully open width (meters)

def normalize_wsg(width):
    return np.clip((width - WSG_MIN) / (WSG_MAX - WSG_MIN), 0.0, 1.0)

def unnormalize_wsg(norm_width):
    return WSG_MIN + norm_width * (WSG_MAX - WSG_MIN)

# TODO: figure out real velocity limit in simulation by closing/opening grippers
WSG_MAX_SPEED = 0.4  # meters/sec width change

def normalize_wsg_velocity(width_velocity):
    v = width_velocity / WSG_MAX_SPEED
    return np.clip(v, -1.0, 1.0)

Run simulation to collect teleoperation data and save it to files.

In [17]:
target_dt = 0.05  # 20 Hz

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant") 

camera_world = station.GetSubsystemByName("rgbd_sensor_camera0")
camera_wrist = station.GetSubsystemByName("rgbd_sensor_camera1")


# add controller
controller = builder.AddSystem(TeleopController(meshcat, plant))

# Connect controller outputs to station inputs
builder.Connect(
    controller.GetOutputPort("iiwa_position_command"),
    station.GetInputPort("iiwa.position")
)
builder.Connect(
    controller.GetOutputPort("wsg_position_command"),
    station.GetInputPort("wsg.position")
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
station_context = diagram.GetSubsystemContext(station, context)

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

print("\n--- Starting Cartesian Teleop Demo ---")
print("Please focus the Meshcat viewer window to enable keyboard input.")


image_data_camera0 = []
image_data_camera1 = []
timestamps = []

qpos_list = []
qvel_list = []
action_list = []  # iiwa + gripper commands


# Run until the user clicks the "Stop Simulation" button in Meshcat
try:
    while meshcat.GetButtonClicks('Stop Simulation') == 0:
        controller.step_control()
        
        simulator.AdvanceTo(simulator.get_context().get_time() + target_dt)

        sim_context = simulator.get_context()
        station_context = diagram.GetSubsystemContext(station, sim_context)

        # ---------- get plant state (qpos, qvel) ----------
        plant_context = station.GetSubsystemContext(plant, station_context)
        
        # Visualize frames for debugging IK
        controller.visualize_frames(plant_context)
        
        iiwa_model = plant.GetModelInstanceByName("iiwa")
        wsg_model  = plant.GetModelInstanceByName("wsg")

        # iiwa joints
        q_iiwa = plant.GetPositions(plant_context, iiwa_model)
        v_iiwa = plant.GetVelocities(plant_context, iiwa_model)

        # gripper joints
        q_wsg = plant.GetPositions(plant_context, wsg_model)
        v_wsg = plant.GetVelocities(plant_context, wsg_model)
        wsg_width = normalize_wsg(q_wsg[1] - q_wsg[0])
        wsg_vel = normalize_wsg_velocity(v_wsg[1] - v_wsg[0])
        # print(v_wsg[1] - v_wsg[0])

        # concatenate to get exactly (7 + 1,) vectors
        qpos = np.concatenate([q_iiwa, [wsg_width]], axis=0).copy()
        qvel = np.concatenate([v_iiwa, [wsg_vel]], axis=0).copy()

        qpos_list.append(qpos)
        qvel_list.append(qvel)

        # ---------- get current action (controller outputs) ----------
        controller_context = diagram.GetSubsystemContext(controller, sim_context)

        iiwa_cmd = controller.GetOutputPort("iiwa_position_command").Eval(controller_context)
        wsg_cmd = controller.GetOutputPort("wsg_position_command").Eval(controller_context)
        # normalize gripper commands
        wsg_cmd = [normalize_wsg(x) for x in wsg_cmd]

        # flatten to 1D arrays and concatenate
        iiwa_cmd_np = np.asarray(iiwa_cmd).ravel()
        wsg_cmd_np = np.asarray(wsg_cmd).ravel()
        action = np.concatenate([iiwa_cmd_np, wsg_cmd_np], axis=0)
        action_list.append(action)

        # Grab the current images from the output ports
        img0 = station.GetOutputPort("camera0.rgb_image").Eval(station_context)  # ImageRgba8U object
        img1 = station.GetOutputPort("camera1.rgb_image").Eval(station_context)  # ImageRgba8U object

        # Convert to numpy arrays for easier handling
        np_img0 = np.array(np.copy(img0.data)).reshape(img0.height(), img0.width(), -1)  # RGBA
        np_img1 = np.array(np.copy(img1.data)).reshape(img1.height(), img1.width(), -1)
        
        # keep only RGB channels; ACT expects (H, W, 3) uint8 images
        image_data_camera0.append(np_img0[:, :, :3].copy())
        image_data_camera1.append(np_img1[:, :, :3].copy())

        timestamps.append(sim_context.get_time())
finally:
    controller.spacemouse.close()

print("Simulation stopped by user.")

SpaceMouse connected via libspnav!

--- Starting Cartesian Teleop Demo ---
Please focus the Meshcat viewer window to enable keyboard input.
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding I

In [54]:
# ---------- convert buffers to numpy arrays ----------
qpos_arr = np.stack(qpos_list, axis=0).astype(np.float32)      # (T, nq)
qvel_arr = np.stack(qvel_list, axis=0).astype(np.float32)      # (T, nq)
action_arr = np.stack(action_list, axis=0).astype(np.float32)  # (T, act_dim)

images0_arr = np.stack(image_data_camera0, axis=0).astype(np.uint8)  # (T, H, W, 3)
images1_arr = np.stack(image_data_camera1, axis=0).astype(np.uint8)  # (T, H, W, 3)

timestamps_arr = np.array(timestamps, dtype=np.float64)

print("qpos: ", qpos_arr, qpos_arr.shape)
print("qvel: ", qvel_arr, qvel_arr.shape)
print("actions: ", action_arr, action_arr.shape)
print(timestamps_arr)

# ---------- write ACT-style HDF5 episode ----------
episode_idx = 0  # you can change this per demonstration
h5_path = os.path.join("teleop_data/sim_open_drawer", f"episode_{episode_idx}.hdf5")

with h5py.File(h5_path, "w") as root:
    # mark this as simulation data (ACT uses this flag)
    root.attrs["sim"] = True

    # actions: shape (T, act_dim)
    root.create_dataset("action", data=action_arr, compression="gzip")

    # observations group
    obs_grp = root.create_group("observations")
    obs_grp.create_dataset("qpos", data=qpos_arr, compression="gzip")
    obs_grp.create_dataset("qvel", data=qvel_arr, compression="gzip")

    # optional: store timestamps (not strictly required by ACT but often useful)
    obs_grp.create_dataset("time", data=timestamps_arr)

    # images subgroup, one dataset per camera
    img_grp = obs_grp.create_group("images")
    img_grp.create_dataset("camera0", data=images0_arr, compression="gzip")
    img_grp.create_dataset("camera1", data=images1_arr, compression="gzip")

print(f"Wrote ACT-style episode to {h5_path}")

qpos:  [[-1.5699646e+00  1.4605041e-01  1.1644751e-04 ...  1.6026086e+00
   9.3513394e-05  9.3241411e-01]
 [-1.5699916e+00  1.8585697e-01  3.7102491e-05 ...  1.6018995e+00
   6.6869732e-05  1.0000000e+00]
 [-1.5699985e+00  1.9698377e-01  8.3490895e-06 ...  1.6009487e+00
   3.3229226e-05  9.9994642e-01]
 ...
 [-1.7540283e+00  7.4309039e-01  9.9437252e-02 ...  2.0318673e+00
   2.3110452e-01  1.0000000e+00]
 [-1.7540283e+00  7.4309039e-01  9.9437252e-02 ...  2.0318673e+00
   2.3110452e-01  1.0000000e+00]
 [-1.7540283e+00  7.4309039e-01  9.9437252e-02 ...  2.0318673e+00
   2.3110451e-01  1.0000000e+00]] (168, 8)
qvel:  [[-7.3472271e-04  1.2501501e+00 -1.6999529e-03 ...  7.1296431e-03
   1.7525586e-04  1.0000000e+00]
 [-2.8226181e-04  4.1819164e-01 -1.0290479e-03 ... -2.1642916e-02
  -7.7249028e-04 -8.7099820e-02]
 [-5.2553965e-05  9.5284596e-02 -2.5850057e-04 ... -1.4544480e-02
  -5.1151495e-04 -5.1108836e-03]
 ...
 [ 1.2257187e-09 -1.3795763e-08 -7.9398921e-10 ... -1.3644613e-07
  -6.6850

Replay collected joint and gripper data. (Need to restart kernel, run first three cells to import dependencies and define scenario_string and meshcat, then run this cell.)

In [6]:
joint_data = np.load("downsampled_teleop_joint_data.npz")
joint_t_uniform = joint_data["time"]
joint_q_uniform = joint_data["q"]

gripper_data = np.load("downsampled_teleop_gripper_data.npz")
gripper_t_uniform = gripper_data["time"]
gripper_q_uniform = gripper_data["q"]

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
plant = station.GetSubsystemByName("plant") 

traj_iiwa = PiecewisePolynomial.FirstOrderHold(joint_t_uniform, joint_q_uniform)
traj_wsg = PiecewisePolynomial.FirstOrderHold(gripper_t_uniform, gripper_q_uniform)

builder = DiagramBuilder()
builder.AddSystem(station)  # your station system already created

# Create trajectory sources
iiwa_source = builder.AddSystem(TrajectorySource(traj_iiwa))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg))

# Connect to the plant
builder.Connect(iiwa_source.get_output_port(0),
                station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(0),
                station.GetInputPort("wsg.position"))

diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

# Run for the duration of the trajectory
simulator.AdvanceTo(joint_t_uniform[-1])

print("Replay finished!")

Replay finished!


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf4fb7c2-18e7-48b8-93e7-63f0209e538b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>